In [1]:
%matplotlib inline
import itertools
import os
os.environ['CUDA_VISIBLE_DEVICES']=""
import numpy as np
import gpflow
import gpflow.training.monitor as mon
import numbers
import matplotlib.pyplot as plt
import tensorflow as tf

/home/mv310/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Demo: `gpflow.training.monitor`
In this notebook we'll demo how to use `gpflow.training.monitor` for logging the optimisation of a GPflow model.

## Creating the GPflow model
We first generate some random data and create a GPflow model.

Under the hood, GPflow gives a unique name to each model which is used to name the Variables it creates in the TensorFlow graph containing a random identifier. This is useful in interactive sessions, where people may create a few models, to prevent variables with the same name conflicting. However, when loading the model, we need to make sure that the names of all the variables are exactly the same as in the checkpoint. This is why we pass name="SVGP" to the model constructor, and why we use gpflow.defer_build().

In [2]:
np.random.seed(0)
X = np.random.rand(10000, 1) * 10
Y = np.sin(X) + np.random.randn(*X.shape)
Xt = np.random.rand(10000, 1) * 10
Yt = np.sin(Xt) + np.random.randn(*Xt.shape)

with gpflow.defer_build():
    m = gpflow.models.SVGP(X, Y, gpflow.kernels.RBF(1), gpflow.likelihoods.Gaussian(),
                           Z=np.linspace(0, 10, 5)[:, None],
                           minibatch_size=100, name="SVGP")
    m.likelihood.variance = 0.01
m.compile()

Let's compute log likelihood before the optimisation

In [3]:
print('LML before the optimisation: %f' % m.compute_log_likelihood())

LML before the optimisation: -1271605.621944


We will be using a TensorFlow optimiser. All TensorFlow optimisers have a support for `global_step` variable. Its purpose is to track how many optimisation steps have occurred. It is useful to keep this in a TensorFlow variable as this allows it to be restored together with all the parameters of the model.

The code below creates this variable using a monitor's helper function. It is important to create it before building the monitor in case the monitor includes a checkpoint task. This is because the checkpoint internally uses the TensorFlow Saver which creates a list of variables to save. Therefore all variables expected to be saved by the checkpoint task should exist by the time the task is created.

In [4]:
session = m.enquire_session()
global_step = mon.create_global_step(session)

## Construct the monitor

Next we need to construct the monitor. `gpflow.training.monitor` provides classes that are building blocks for the monitor. Essengially, a monitor is a function that is provided as a callback to an optimiser. It consists of a number of tasks that may be executed at each step, subject to their running condition.

In this example, we want to:
- log certain scalar parameters in TensorBoard,
- log the full optimisation objective (log marginal likelihood bound) periodically, even though we optimise with minibatches,
- store a backup of the optimisation process periodically,
- log performance for a test set periodically.

We will define these tasks as follows:

In [5]:
print_task = mon.PrintTimingsTask().with_name('print')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

sleep_task = mon.SleepTask(0.01).with_name('sleep').with_name('sleep')

saver_task = mon.CheckpointTask('./monitor-saves').with_name('saver')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

file_writer = mon.LogdirWriter('./model-tensorboard')

model_tboard_task = mon.ModelToTensorBoardTask(file_writer, m).with_name('model_tboard')\
    .with_condition(mon.PeriodicIterationCondition(10))\
    .with_exit_condition(True)

lml_tboard_task = mon.LmlToTensorBoardTask(file_writer, m).with_name('lml_tboard')\
    .with_condition(mon.PeriodicIterationCondition(100))\
    .with_exit_condition(True)

As the above code shows, each task can be assigned a name and running conditions. The name will be shown in the task timing summary.

There are two different types of running conditions: `with_condition` controls execution of the task at each iteration in the optimisation loop. `with_exit_condition` is a simple boolean flag indicating that the task should also run at the end of optimisation.
In this example we want to run our tasks periodically, at every iteration or every 10th or 100th iteration.

Notice that the two TensorBoard tasks will write events into the same file. It is possible to share a file writer between multiple tasks. However it is not possible to share the same event location between multiple file writers. An attempt to open two writers with the same location will result in error.


## Custom tasks
We may also want to perfom certain tasks that do not have pre-defined `Task` classes. For example, we may want to compute the performance on a test set. Here we create such a class by extending `BaseTensorBoardTask` to log the testing benchmarks in addition to all the scalar parameters.

In [6]:
class CustomTensorBoardTask(mon.BaseTensorBoardTask):
    def __init__(self, file_writer, model, Xt, Yt):
        super().__init__(file_writer, model)
        self.Xt = Xt
        self.Yt = Yt
        self._full_test_err = tf.placeholder(gpflow.settings.float_type, shape=())
        self._full_test_nlpp = tf.placeholder(gpflow.settings.float_type, shape=())
        self._summary = tf.summary.merge([tf.summary.scalar("test_rmse", self._full_test_err),
                                         tf.summary.scalar("test_nlpp", self._full_test_nlpp)])
    
    def run(self, context: mon.MonitorContext, *args, **kwargs) -> None:
        minibatch_size = 100
        preds = np.vstack([self.model.predict_y(Xt[mb * minibatch_size:(mb + 1) * minibatch_size, :])[0]
                            for mb in range(-(-len(Xt) // minibatch_size))])
        test_err = np.mean((Yt - preds) ** 2.0)**0.5
        self._eval_summary(context, {self._full_test_err: test_err, self._full_test_nlpp: 0.0})

        
custom_tboard_task = CustomTensorBoardTask(file_writer, m, Xt, Yt).with_name('custom_tboard')\
    .with_condition(mon.PeriodicIterationCondition(100))\
    .with_exit_condition(True)

Now we can put all these tasks into a monitor.

In [7]:
monitor_tasks = [print_task, model_tboard_task, lml_tboard_task, custom_tboard_task, saver_task, sleep_task]
monitor = mon.Monitor(monitor_tasks, session, global_step)

## Running the optimisation
We finally get to running the optimisation.

We may want to continue a previously run optimisation by resotring the TensorFlow graph from the latest checkpoint. Otherwise skip this step.

In [8]:
if os.path.isdir('./monitor-saves'):
    mon.restore_session(session, './monitor-saves')

In [9]:
optimiser = gpflow.train.AdamOptimizer(0.01)

with mon.Monitor(monitor_tasks, session, global_step, print_summary=True) as monitor:
    optimiser.minimize(m, step_callback=monitor, maxiter=450, global_step=global_step)

file_writer.close()

Iteration 10	total itr.rate 12.02/s	recent itr.rate 12.02/s	opt.step 10	total opt.rate 13.49/s	recent opt.rate 13.49/s
Iteration 20	total itr.rate 17.87/s	recent itr.rate 34.80/s	opt.step 20	total opt.rate 26.54/s	recent opt.rate 820.59/s
Iteration 30	total itr.rate 22.46/s	recent itr.rate 46.27/s	opt.step 30	total opt.rate 38.90/s	recent opt.rate 561.37/s
Iteration 40	total itr.rate 25.89/s	recent itr.rate 47.68/s	opt.step 40	total opt.rate 50.75/s	recent opt.rate 592.97/s
Iteration 50	total itr.rate 28.44/s	recent itr.rate 46.94/s	opt.step 50	total opt.rate 62.08/s	recent opt.rate 580.30/s
Iteration 60	total itr.rate 30.37/s	recent itr.rate 45.99/s	opt.step 60	total opt.rate 72.80/s	recent opt.rate 530.95/s
Iteration 70	total itr.rate 31.95/s	recent itr.rate 46.39/s	opt.step 70	total opt.rate 83.31/s	recent opt.rate 625.60/s
Iteration 80	total itr.rate 33.26/s	recent itr.rate 46.73/s	opt.step 80	total opt.rate 93.33/s	recent opt.rate 588.08/s


  0%|          | 0/100 [00:00<?, ?it/s]

Iteration 90	total itr.rate 34.27/s	recent itr.rate 45.25/s	opt.step 90	total opt.rate 102.96/s	recent opt.rate 591.78/s
Iteration 100	total itr.rate 35.43/s	recent itr.rate 50.97/s	opt.step 100	total opt.rate 113.15/s	recent opt.rate 1032.42/s


100%|██████████| 100/100 [00:00<00:00, 398.93it/s]


Iteration 110	total itr.rate 30.03/s	recent itr.rate 11.91/s	opt.step 110	total opt.rate 122.28/s	recent opt.rate 632.37/s
Iteration 120	total itr.rate 30.81/s	recent itr.rate 43.03/s	opt.step 120	total opt.rate 129.67/s	recent opt.rate 387.68/s
Iteration 130	total itr.rate 31.56/s	recent itr.rate 44.48/s	opt.step 130	total opt.rate 138.16/s	recent opt.rate 643.12/s
Iteration 140	total itr.rate 32.36/s	recent itr.rate 48.50/s	opt.step 140	total opt.rate 147.09/s	recent opt.rate 920.47/s
Iteration 150	total itr.rate 33.22/s	recent itr.rate 52.80/s	opt.step 150	total opt.rate 156.02/s	recent opt.rate 1041.22/s
Iteration 160	total itr.rate 34.03/s	recent itr.rate 53.58/s	opt.step 160	total opt.rate 164.78/s	recent opt.rate 1044.40/s
Iteration 170	total itr.rate 34.60/s	recent itr.rate 47.35/s	opt.step 170	total opt.rate 172.78/s	recent opt.rate 772.81/s
Iteration 180	total itr.rate 35.05/s	recent itr.rate 45.10/s	opt.step 180	total opt.rate 179.82/s	recent opt.rate 585.96/s
Iteration 190	

...

Iteration 410	total itr.rate 36.13/s	recent itr.rate 14.87/s	opt.step 410	total opt.rate 290.23/s	recent opt.rate 377.88/s
Iteration 420	total itr.rate 36.30/s	recent itr.rate 45.32/s	opt.step 420	total opt.rate 294.90/s	recent opt.rate 865.01/s
Iteration 430	total itr.rate 36.53/s	recent itr.rate 49.97/s	opt.step 430	total opt.rate 298.60/s	recent opt.rate 631.63/s
Iteration 440	total itr.rate 36.78/s	recent itr.rate 51.44/s	opt.step 440	total opt.rate 302.12/s	recent opt.rate 613.32/s
Iteration 450	total itr.rate 36.93/s	recent itr.rate 44.99/s	opt.step 450	total opt.rate 305.26/s	recent opt.rate 562.42/s


 46%|████▌     | 46/100 [00:00<00:00, 459.75it/s]

Iteration 450	total itr.rate 36.00/s	recent itr.rate 0.00/s	opt.step 450	total opt.rate 268.14/s	recent opt.rate 0.00/s


100%|██████████| 100/100 [00:00<00:00, 466.34it/s]


Tasks execution time summary:
print:	0.0385 (sec)
model_tboard:	0.2123 (sec)
lml_tboard:	1.2115 (sec)
custom_tboard:	1.2399 (sec)
saver:	4.0557 (sec)
sleep:	4.5382 (sec)


Now lets compute the log likelihood again. Hopefully we will see an increase in its value

In [10]:
print('LML after the optimisation: %f' % m.compute_log_likelihood())

LML after the optimisation: -68705.124191


In this example we have used the TensorFlow `AdamOptimizer`. Using `ScipyOptimizer` requires a couple of special tricks. Firstly, this optimiser works with its own copy of trained variables and updates the original ones only when the optimisation is completed. Secondly, it doesn't use the `global_step` variable. This can present a problem when doing optimisation in several stages. Monitor has to use an iteration count instead of the `global_step`, which will be reset to zero at each stage.

To adress the first problem we will provide the optimiser as one of the parameters to the monitor. The monitor will make sure the orginal variables are updated whenever we access them from a monitoring task. The second problem is addressed by creating an instance of `MonitorContext` and providing it explicitely to the `Monitor`.


In [11]:
optimiser = gpflow.train.ScipyOptimizer()
context = mon.MonitorContext()

with mon.Monitor([print_task], session, print_summary=True, optimiser=optimiser, context=context) as monitor:
    optimiser.minimize(m, step_callback=monitor, maxiter=250)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15576.597541
  Number of iterations: 5
  Number of functions evaluations: 12


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 15576.597541
  Number of iterations: 5
  Number of functions evaluations: 12


Iteration 5	total itr.rate 5.46/s	recent itr.rate nan/s	opt.step 5	total opt.rate 5.46/s	recent opt.rate nan/s
Tasks execution time summary:
print:	0.0386 (sec)
